In [1]:
import time
import sys,os
import requests
import numpy as np
import pandas as pd
from ta.momentum import StochasticOscillator

from FinMind import strategies
from FinMind.data import DataLoader
from FinMind.strategies.base import Strategy
from FinMind.data import FinMindApi

stock_id_ignored = ['0054', '0058', '0059', '0060', '00625K', '00643K', '00649', '00658L', '00659R', '00667', '00672L', '00677U', '00691R', '00698L', '00699R', '00704L', '00705R', '00716R', '00729R', '00732', '00742', '00743', '00766L', '00767', '00774B', '00774C', '00776', '0080', '0081', '008201', '00866', '00906', '00938', '00943', '00944', '00945B', '00946', '00947', '00949', '00951', '00952', '00953B', '00954', '00956', '00960', '00961', '00963', '00964', '00965']
stock_id_available = ['0050', '0051', '0052', '0053', '0055', '0056', '0057', '0061', '006203', '006204', '006205', '006206', '006207', '006208', '00631L', '00632R', '00633L', '00634R', '00635U', '00636', '00636K', '00637L', '00638R', '00639', '00640L', '00641R', '00642U', '00643', '00645', '00646', '00647L', '00648R', '00650L', '00651R', '00652', '00653L', '00654R', '00655L', '00656R', '00657', '00657K', '00660', '00661', '00662', '00663L', '00664R', '00665L', '00666R', '00668', '00668K', '00669R', '00670L', '00671R', '00673R', '00674R', '00675L', '00676R', '00678', '00680L', '00681R', '00682U', '00683L', '00684R', '00685L', '00686R', '00688L', '00689R', '00690', '00692', '00693U', '00700', '00701', '00702', '00703', '00706L', '00707R', '00708L', '00709', '00710B', '00711B', '00712', '00713', '00714', '00715L', '00717', '00728', '00730', '00731', '00733', '00735', '00736', '00737', '00738U', '00739', '00752', '00753L', '00757', '00762', '00763U', '00770', '00771', '00775B', '00783', '00830', '00850', '00851', '00852L', '00861', '00865B', '00875', '00876', '00878', '00881', '00882', '00885', '00891', '00892', '00893', '00894', '00895', '00896', '00897', '00898', '00899', '00900', '00901', '00902', '00903', '00904', '00905', '00907', '00908', '00909', '00910', '00911', '00912', '00913', '00915', '00916', '00917', '00918', '00919', '00920', '00921', '00922', '00923', '00924', '00925', '00926', '00927', '00929', '00930', '00932', '00934', '00935', '00936', '00939', '00940', '00941']

In [2]:
# update token from a file
token_path = "./token"
token = ""
with open(token_path, "r") as fp:
    token = fp.read()
    token = token.replace("\n", "")
if len(token) <= 100:
    print("ERROR: need a token from FinMind")
print(f"token length:{len(token)}")

# get account status
url = "https://api.web.finmindtrade.com/v2/user_info"
payload = {
    "token": token+"1",
}
resp = requests.get(url, params=payload)
print("使用次數:", resp.json()["user_count"])  # 使用次數
print("api 使用上限", resp.json()["api_request_limit"])  # api 使用上限
print(resp.status_code)
print(resp.json())

token length:201
使用次數: 0
api 使用上限 6000
200
{'msg': 'success', 'status': 200, 'email_verify': True, 'user_id': 'raulhsiao', 'email': 'raulhsiao@gmail.com', 'level': 3, 'end_date': '', 'user_count': 0, 'api_request_limit': 6000, 'api_request_limit_hour': 6000, 'api_request_limit_day': '-', 'level_title': 'Sponsor', 'BackerInfo': {'subscription_begin_date': '', 'subscription_expired_date': '', 'api_request_limit': 0, 'msg': '', 'status_code': 200}, 'SponsorInfo': {'subscription_begin_date': '2025-02-03', 'subscription_expired_date': '2026-03-04', 'api_request_limit': 6000, 'msg': '', 'status_code': 200}, 'SponsorProInfo': {'subscription_begin_date': '', 'subscription_expired_date': '', 'api_request_limit': 0, 'msg': '', 'status_code': 200}}


In [4]:
api = DataLoader()
df = api.taiwan_stock_info()
df

2025-07-28 20:03:35.103 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockInfo, data_id: 


,industry_category,stock_id,stock_name,type,date
0,電子零組件業,5481,新華,tpex,2020-06-03
1,文化創意業,3687,歐買尬,tpex,2020-06-03
2,光電業,3629,地心引力,tpex,2020-06-03
3,電腦及週邊設備業,5450,寶聯通,tpex,2020-06-03
4,其他電子類,6238,勝麗,tpex,2020-06-13
...,...,...,...,...,...
3942,Index,BiotechnologyMedicalCare,生技醫療類指數,twse,None
3943,Index,Automobile,汽車類指數,twse,None
3944,大盤,TPEx,櫃買指數,tpex,None
3945,Index,Food,食品類指數,twse,None


In [5]:
tw_stock_info_df = api.taiwan_stock_info()
stock_list_df = tw_stock_info_df.loc[tw_stock_info_df["industry_category"]=="ETF"]
for index,row in stock_list_df.iterrows():
    stock_id = row["stock_id"]
    stock_name = row["stock_name"]
    
    if stock_id in stock_id_ignored:
        continue
        
    df = api.taiwan_stock_daily(
        stock_id=row["stock_id"],
        start_date='2024-04-02',
        end_date='2024-04-30'
    )
    if df.size > 0:
        # print(row["stock_id"], row["stock_name"], row["date"], df.size)
        stock_id_available.append(row["stock_id"])
    else :
        print("BYPASS", )
        stock_id_ignored.append(row["stock_id"])
        

2025-07-28 20:03:37.156 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockInfo, data_id: 
2025-07-28 20:03:37.692 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00925
2025-07-28 20:03:37.859 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 0050
2025-07-28 20:03:38.032 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00783
2025-07-28 20:03:38.284 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00830
2025-07-28 20:03:38.457 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00775B
2025-07-28 20:03:38.643 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00771
2025-07-28 20:03:38.889 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_

BYPASS


2025-07-28 20:04:22.907 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00916
2025-07-28 20:04:23.300 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00917
2025-07-28 20:04:23.477 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00918
2025-07-28 20:04:23.722 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00919
2025-07-28 20:04:23.966 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00920
2025-07-28 20:04:24.206 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00921
2025-07-28 20:04:24.397 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00922
2025-07-28 20:04:24.764 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice,

BYPASS


2025-07-28 20:04:28.144 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 009800


...

BYPASS


2025-07-28 20:04:31.847 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00984A


BYPASS


2025-07-28 20:04:32.084 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00985A
2025-07-28 20:04:32.250 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 009810


BYPASS
BYPASS
BYPASS


In [6]:
for i in stock_id_available:
    print(i) 

0050
0051
0052
0053
0055
0056
0057
0061
006203
006204
006205
006206
006207
006208
00631L
00632R
00633L
00634R
00635U
00636
00636K
00637L
00638R
00639
00640L
00641R
00642U
00643
00645
00646
00647L
00648R
00650L
00651R
00652
00653L
00654R
00655L
00656R
00657
00657K
00660
00661
00662
00663L
00664R
00665L
00666R
00668
00668K
00669R
00670L
00671R
00673R
00674R
00675L
00676R
00678
00680L
00681R
00682U
00683L
00684R
00685L
00686R
00688L
00689R
00690
00692
00693U
00700
00701
00702
00703
00706L
00707R
00708L
00709
00710B
00711B
00712
00713
00714
00715L
00717
00728
00730
00731
00733
00735
00736
00737
00738U
00739
00752
00753L
00757
00762
00763U
00770
00771
00775B
00783
00830
00850
00851
00852L
00861
00865B
00875
00876
00878
00881
00882
00885
00891
00892
00893
00894
00895
00896
00897
00898
00899
00900
00901
00902
00903
00904
00905
00907
00908
00909
00910
00911
00912
00913
00915
00916
00917
00918
00919
00920
00921
00922
00923
00924
00925
00926
00927
00929
00930
00932
00934
00935
00936
00939
00940
